<center>
<img src="https://raw.githubusercontent.com/Yorko/mlcourse.ai/master/img/ods_stickers.jpg" />
    
## [mlcourse.ai](https://mlcourse.ai) - Open Machine Learning Course

<center>
Auteur: [Egor Polusmak](https://www.linkedin.com/in/egor-polusmak/).  
Traduit et édité par [Yuanyuan Pao](https://www.linkedin.com/in/yuanyuanpao/) et [Ousmane Cissé](https://fr.linkedin.com/in/ousmane-cisse).  
Ce matériel est soumis aux termes et conditions de la licence [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/).   
L'utilisation gratuite est autorisée à des fins non commerciales.

# <center>Topic 9. Analyse des séries temporelles en Python</center>
## <center>Partie 2. Prédire l'avenir avec Facebook Prophet</center>

La prévision de séries chronologiques trouve une large application dans l'analyse de données. Ce ne sont que quelques-unes des prévisions imaginables des tendances futures qui pourraient être utiles:
- Le nombre de serveurs dont un service en ligne aura besoin l'année prochaine.
- La demande d'un produit d'épicerie dans un supermarché un jour donné.
- Le cours de clôture de demain d'un actif financier négociable.

Pour un autre exemple, nous pouvons faire une prédiction des performances d'une équipe, puis l'utiliser comme référence: d'abord pour fixer des objectifs pour l'équipe, puis pour mesurer les performances réelles de l'équipe par rapport à la référence.

Il existe plusieurs méthodes différentes pour prédire les tendances futures, par exemple, [ARIMA](https://en.wikipedia.org/wiki/Autoregressive_integrated_moving_average), [ARCH](https://en.wikipedia.org/wiki/Autoregressive_conditional_heteroskedasticity), [modèles régressifs](https://en.wikipedia.org/wiki/Autoregressive_model), [réseaux de neurones](https://medium.com/machine-learning-world/neural-networks-for-algorithmic-trading-1-2-correct-time-series-forecasting-backtesting-9776bfd9e589).

Dans cet article, nous examinerons [Prophet](https://facebook.github.io/prophet/), une bibliothèque de prévisions de séries chronologiques publiée par Facebook et open source, le 23 février 2017. Nous l'essayerons également dans le problème de prédiction du nombre quotidien de publications sur Medium.

## Plan de l'article

1. Introduction
2. Le modèle de prévision de Prophet
3. Entraînez-vous avec le Prophet
    * 3.1 Installation en Python
    * 3.2 Ensemble de données
    * 3.3 Analyse visuelle exploratoire
    * 3.4 Faire une prévision
    * 3.5 Évaluation de la qualité des prévisions
    * 3.6 Visualisation
4. Transformation Box-Cox
5. Résumé
6. Références

## 1. Introduction

Selon [l'article](https://research.fb.com/prophet-forecasting-at-scale/) sur Facebook Research, Prophet a été initialement développé dans le but de créer des prévisions commerciales de haute qualité. Cette bibliothèque tente de résoudre les difficultés suivantes, communes à de nombreuses séries chronologiques commerciales:
- Effets saisonniers causés par le comportement humain: cycles hebdomadaires, mensuels et annuels, creux et pics les jours fériés.
- Changements de tendance dus aux nouveaux produits et aux événements du marché.
- Valeurs aberrantes.

Les auteurs affirment que, même avec les paramètres par défaut, dans de nombreux cas, leur bibliothèque produit des prévisions aussi précises que celles fournies par des analystes expérimentés.

De plus, Prophet dispose d'un certain nombre de personnalisations intuitives et facilement interprétables qui permettent d'améliorer progressivement la qualité du modèle de prévision. Ce qui est particulièrement important, ces paramètres sont tout à fait compréhensibles même pour les non-experts en analyse de séries chronologiques, qui est un domaine de la science des données nécessitant certaines compétences et expérience.

Soit dit en passant, l'article d'origine s'intitule «Prévisions à grande échelle», mais il ne s'agit pas de l'échelle au sens «habituel», qui traite des problèmes de calcul et d'infrastructure d'un grand nombre de programmes de travail. Selon les auteurs, Prophet devrait bien évoluer dans les 3 domaines suivants:
- Accessibilité à un large public d'analystes, éventuellement sans expertise approfondie des séries chronologiques.
- Applicabilité à un large éventail de problèmes de prévision distincts.
- Estimation automatisée des performances d'un grand nombre de prévisions, y compris la signalisation des problèmes potentiels pour leur inspection ultérieure par l'analyste.

## 2. Le modèle de prévision Prophet

Maintenant, regardons de plus près comment fonctionne Prophet. Dans son essence, cette bibliothèque utilise le [modèle de régression additive](https://en.wikipedia.org/wiki/Additive_model) $y(t)$ comprenant les composants suivants:

$$y(t) = g(t) + s(t) + h(t) + \epsilon_{t},$$

où:
* La tendance $g(t)$ modélise les changements non périodiques.
* La saisonnalité $s(t)$ représente des changements périodiques.
* La composante vacances $h(t)$ fournit des informations sur les vacances et les événements.

Ci-dessous, nous considérerons quelques propriétés importantes de ces composants de modèle.

### Tendance

La bibliothèque Prophet implémente deux modèles de tendance possibles pour $g(t)$.

Le premier est appelé *Croissance saturée non linéaire*. Il est représenté sous la forme du [modèle de croissance logistique](https://en.wikipedia.org/wiki/Fonction_logistique):

$$g(t) = \frac{C}{1+e^{-k(t - m)}},$$

où:

* $C$ est la capacité de charge (c'est-à-dire la valeur maximale de la courbe).

* $k$ est le taux de croissance (qui représente "la pente" de la courbe).

* $m$ est un paramètre de décalage.

Cette équation logistique permet de modéliser la croissance non linéaire avec saturation, c'est-à-dire lorsque le taux de croissance d'une valeur diminue avec sa croissance. Un des exemples typiques serait de représenter la croissance de l'audience d'une application ou d'un site Web.

En fait, $C$ et $k$ ne sont pas nécessairement des constantes et peuvent varier dans le temps. Prophet prend en charge le réglage automatique et manuel de leur variabilité. La bibliothèque peut elle-même choisir des points optimaux de changements de tendance en ajustant les données historiques fournies.

En outre, Prophet permet aux analystes de définir manuellement des points de changement du taux de croissance et des valeurs de capacité à différents moments. Par exemple, les analystes peuvent avoir des informations sur les dates des versions précédentes qui ont influencé de manière importante certains indicateurs clés de produit.

Le deuxième modèle de tendance est un simple *modèle linéaire par morceaux* (Piecewise Linear Model) avec un taux de croissance constant.  
Il est le mieux adapté aux problèmes sans saturation de la croissance.

### Saisonnalité

La composante saisonnière $s(t)$ fournit un modèle flexible de changements périodiques dus à la saisonnalité hebdomadaire et annuelle.

Les données saisonnières hebdomadaires sont modélisées avec des variables factices. Six nouvelles variables sont ajoutées: «lundi», «mardi», «mercredi», «jeudi», «vendredi», «samedi», qui prennent des valeurs 0 ou 1 selon le jour de la semaine. La caractéristique «dimanche» n'est pas ajoutée car ce serait une combinaison linéaire des autres jours de la semaine, et ce fait aurait un effet négatif sur le modèle.

Le modèle de saisonnalité annuelle dans Prophet repose sur la série de Fourier.

Depuis la version 0.2, vous pouvez également utiliser des séries chronologiques infra-journalières et faire des prévisions infra-journalières, ainsi qu'utiliser la nouvelle caractéristique de saisonnalité quotidienne.

### Vacances et événements

La composante $h(t)$ représente les jours anormaux prévisibles de l'année, y compris ceux dont les horaires sont irréguliers, par exemple les Black Fridays.

Pour utiliser cette caractéristique, l'analyste doit fournir une liste personnalisée d'événements.

### Erreur

Le terme d'erreur $\epsilon(t)$ représente des informations qui n'étaient pas reflétées dans le modèle. Habituellement, il est modélisé comme un bruit normalement distribué.

### Analyse comparative (benchmark) de Prophet

Pour une description détaillée du modèle et des algorithmes derrière Prophet, reportez-vous à l'article ["Forecasting at scale"](https://peerj.com/preprints/3190/) de Sean J. Taylor et Benjamin Letham.

Les auteurs ont également comparé leur bibliothèque avec plusieurs autres méthodes de prévision de séries chronologiques. Ils ont utilisé l'[Erreur absolue moyenne en pourcentage (MAPE)](https://en.wikipedia.org/wiki/Mean _absolue_ pourcentage_erreur) comme mesure de la précision de la prédiction. Dans cette analyse, Prophet a montré une erreur de prévision considérablement plus faible que les autres modèles.

<img src="https://raw.githubusercontent.com/Yorko/mlcourse.ai/master/img/topic9_benchmarking_prophet.png" />

Regardons de plus près comment la qualité de la prévision a été mesurée dans l'article. Pour ce faire, nous aurons besoin de la formule d'erreur moyenne absolue en pourcentage.

Soit $y_{i}$ la *valeur réelle (historique)* et $\hat{y}_{i}$ la *valeur prévue* donnée par notre modèle.

$e_{i} = y_{i} - \hat{y}_{i}$ est alors *l'erreur de prévision* et $p_{i} =\frac{\displaystyle e_{i}}{\displaystyle y_{i}}$ est *l'erreur de prévision relative*.

Nous définissons

$$MAPE = mean\big(\left |p_{i} \right |\big)$$

MAPE est largement utilisé comme mesure de la précision des prédictions car il exprime l'erreur en pourcentage et peut donc être utilisé dans les évaluations de modèles sur différents ensembles de données.

De plus, lors de l'évaluation d'un algorithme de prévision, il peut s'avérer utile de calculer [MAE (Mean Absolute Error)](https://en.wikipedia.org/wiki/Mean _error_ absolue) afin d'avoir une image des erreurs en nombres absolus. En utilisant des composants précédemment définis, son équation sera

$$MAE = mean\big(\left |e_{i}\right |\big)$$

Quelques mots sur les algorithmes avec lesquels Prophet a été comparé. La plupart d'entre eux sont assez simples et sont souvent utilisés comme référence pour d'autres modèles:
* `naive` est une approche de prévision simpliste dans laquelle nous prédisons toutes les valeurs futures en nous appuyant uniquement sur l'observation au dernier moment disponible.
* `snaive` (saisonnier naïf) est un modèle qui fait des prédictions constantes en tenant compte des informations sur la saisonnalité. Par exemple, dans le cas de données saisonnières hebdomadaires pour chaque futur lundi, nous prédirions la valeur du dernier lundi et pour tous les futurs mardis, nous utiliserions la valeur du dernier mardi, etc.
* `mean` utilise la valeur moyenne des données comme prévision.
* `arima` signifie *Autoregressive Integrated Moving Average*, voir [Wikipedia](https://en.wikipedia.org/wiki/Autoregressive_integrated_moving_average) pour plus de détails.
* `ets` signifie *Lissage exponentiel*, voir [Wikipedia](https://en.wikipedia.org/wiki/Exponential_smoothing) pour plus d'informations.

## 3. Entraînez-vous avec Facebook Prophet

### 3.1 Installation en Python

Tout d'abord, vous devez installer la bibliothèque. Prophet est disponible pour Python et R. Le choix dépendra de vos préférences personnelles et des exigences du projet. Plus loin dans cet article, nous utiliserons Python.

En Python, vous pouvez installer Prophet à l'aide de PyPI:
```
$ pip install fbprophet
```

Dans R, vous trouverez le package CRAN correspondant. Reportez-vous à la [documentation](https://facebookincubator.github.io/prophet/docs/installation.html) pour plus de détails.

Importons les modules dont nous aurons besoin et initialisons notre environnement:

In [ ]:
import warnings

warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy import stats

%matplotlib inline

### 3.2 Jeu de données

Nous prédirons le nombre quotidien de publications publiées sur [Medium](https://medium.com/).

Tout d'abord, nous chargeons notre jeu de données.

In [ ]:
df = pd.read_csv("../../data/medium_posts.csv.zip", sep="\t")

Ensuite, nous omettons toutes les colonnes à l'exception de `published` et `url`. Le premier correspond à la dimension temporelle tandis que le second identifie de manière unique un message par son URL. Par la suite, nous nous débarrassons des doublons possibles et des valeurs manquantes dans les données:

In [ ]:
df = df[["published", "url"]].dropna().drop_duplicates()

Ensuite, nous devons convertir `published` au format datetime car par défaut `pandas` traite ce champ comme une chaîne.

In [ ]:
df["published"] = pd.to_datetime(df["published"])

Trions la trame de données par date et jetons un œil à ce que nous avons:

In [ ]:
df.sort_values(by=["published"]).head(n=3)

La date de sortie publique de Medium était le 15 août 2012. Mais, comme vous pouvez le voir sur les données ci-dessus, il existe au moins plusieurs lignes avec des dates de publication beaucoup plus anciennes. Ils sont apparus d'une manière ou d'une autre dans notre ensemble de données, mais ils ne sont guère légitimes. Nous allons simplement couper notre série chronologique pour ne conserver que les lignes qui tombent sur la période du 15 août 2012 au 25 juin 2017:

In [ ]:
df = df[
    (df["published"] > "2012-08-15") & (df["published"] < "2017-06-26")
].sort_values(by=["published"])
df.head(n=3)

In [ ]:
df.tail(n=3)

Comme nous allons prédire le nombre de publications, nous allons agréger et compter les publications uniques à chaque moment donné. Nous nommerons la nouvelle colonne correspondante `posts`:

In [ ]:
aggr_df = df.groupby("published")[["url"]].count()
aggr_df.columns = ["posts"]

Dans cette pratique, nous sommes intéressés par le nombre de messages **par jour**. Mais en ce moment, toutes nos données sont divisées en intervalles de temps irréguliers qui sont inférieurs à une journée. C'est ce qu'on appelle une série chronologique infra-journalière (*sub-daily time series*). Pour le voir, affichons les 3 premières lignes:

In [ ]:
aggr_df.head(n=3)

Pour résoudre ce problème, nous devons agréger le nombre de messages par "bins" d'une taille de date. Dans l'analyse des séries chronologiques, ce processus est appelé *rééchantillonnage* (*resampling*). Et si l'on *réduit* le taux d'échantillonnage des données, il est souvent appelé *sous-échantillonnage* (*downsampling*).

Heureusement, `pandas` a une fonctionnalité intégrée pour cette tâche. Nous allons rééchantillonner notre indice de date jusqu'à des "bins" d'un jour:

In [ ]:
daily_df = aggr_df.resample("D").apply(sum)
daily_df.head(n=3)

### 3.3 Analyse visuelle exploratoire

Comme toujours, il peut être utile et instructif de regarder une représentation graphique de vos données.

Nous allons créer un tracé de série chronologique pour toute la plage de temps. L'affichage de données sur une période aussi longue peut donner des indices sur la saisonnalité et les écarts anormaux visibles.

Tout d'abord, nous importons et initialisons la bibliothèque `Plotly`, qui permet de créer de superbes graphes interactifs:

In [ ]:
from plotly import graph_objs as go
from plotly.offline import init_notebook_mode, iplot

# Initialize plotly
init_notebook_mode(connected=True)

Nous définissons également une fonction d'aide, qui tracera nos trames de données tout au long de l'article:

In [ ]:
def plotly_df(df, title=""):
    """Visualize all the dataframe columns as line plots."""
    common_kw = dict(x=df.index, mode="lines")
    data = [go.Scatter(y=df[c], name=c, **common_kw) for c in df.columns]
    layout = dict(title=title)
    fig = dict(data=data, layout=layout)
    iplot(fig, show_link=False)

Essayons de tracer notre jeu de données *tel quel*:

In [ ]:
plotly_df(daily_df, title="Posts on Medium (daily)")

Les données à haute fréquence peuvent être assez difficiles à analyser. Même avec la possibilité de zoomer fournie par `Plotly`, il est difficile d'inférer quoi que ce soit de significatif à partir de ce graphique, à l'exception de la tendance à la hausse et à l'accélération.

Pour réduire le bruit, nous allons rééchantillonner le compte à rebours des postes jusqu'à la semaine. Outre le *binning*, d'autres techniques possibles de réduction du bruit incluent [Moving-Average Smoothing](https://en.wikipedia.org/wiki/Moving_average) et [Exponential Smoothing](https://en.wikipedia.org/wiki/Exponential_smoothing), entre autres.

Nous sauvegardons notre dataframe sous-échantillonné dans une variable distincte, car dans cette pratique, nous ne travaillerons qu'avec des séries journalières:

In [ ]:
weekly_df = daily_df.resample("W").apply(sum)

Enfin, nous traçons le résultat:

In [ ]:
plotly_df(weekly_df, title="Posts on Medium (weekly)")

Ce graphique sous-échantillonné s'avère un peu meilleur pour la perception d'un analyste.

L'une des fonctions les plus utiles fournies par `Plotly` est la possibilité de plonger rapidement dans différentes périodes de la chronologie afin de mieux comprendre les données et trouver des indices visuels sur les tendances possibles, les effets périodiques et irréguliers.

Par exemple, un zoom avant sur quelques années consécutives nous montre des points temporels correspondant aux vacances de Noël, qui influencent grandement les comportements humains.

Maintenant, nous allons omettre les premières années d'observations, jusqu'en 2015. Premièrement, elles ne contribueront pas beaucoup à la qualité des prévisions en 2017. Deuxièmement, ces premières années, ayant un nombre très faible de messages par jour, sont susceptible d'augmenter le bruit dans nos prévisions, car le modèle serait obligé d'ajuster ces données historiques anormales avec des données plus pertinentes et indicatives des dernières années.

In [ ]:
daily_df = daily_df.loc[daily_df.index >= "2015-01-01"]
daily_df.head(n=3)

Pour résumer, à partir de l'analyse visuelle, nous pouvons voir que notre ensemble de données n'est pas stationnaire avec une tendance croissante importante. Il montre également une saisonnalité hebdomadaire et annuelle et un certain nombre de jours anormaux chaque année.

### 3.4 Faire une prévision

L'API de Prophet est très similaire à celle que vous pouvez trouver dans `sklearn`. Nous créons d'abord un modèle, puis appelons la méthode `fit` et, enfin, faisons une prévision. L'entrée de la méthode `fit` est un` DataFrame` avec deux colonnes:
* `ds` (datestamp ou horodatage) doit être de type` date` ou `datetime`.
* `y` est une valeur numérique que nous voulons prédire.

Pour commencer, nous allons importer la bibliothèque et éliminer les messages de diagnostic sans importance:

In [ ]:
import logging

from fbprophet import Prophet

logging.getLogger().setLevel(logging.ERROR)

Convertissons notre dataframe de données au format requis par Prophet:

In [ ]:
df = daily_df.reset_index()
df.columns = ["ds", "y"]
df.tail(n=3)

Les auteurs de la bibliothèque conseillent généralement de faire des prédictions basées sur au moins plusieurs mois, idéalement, plus d'un an de données historiques. Heureusement, dans notre cas, nous avons plus de quelques années de données pour s'adapter au modèle.

Pour mesurer la qualité de nos prévisions, nous devons diviser notre ensemble de données en une *partie historique*, qui est la première et la plus grande tranche de nos données, et une *partie prédiction*, qui sera située à la fin de la chronologie. Nous allons supprimer le dernier mois de l'ensemble de données afin de l'utiliser plus tard comme cible de prédiction:

In [ ]:
prediction_size = 30
train_df = df[:-prediction_size]
train_df.tail(n=3)

Maintenant, nous devons créer un nouvel objet `Prophet`. Ici, nous pouvons passer les paramètres du modèle dans le constructeur. Mais dans cet article, nous utiliserons les valeurs par défaut. Ensuite, nous formons notre modèle en invoquant sa méthode `fit` sur notre jeu de données de formation:

In [ ]:
m = Prophet()
m.fit(train_df);

En utilisant la méthode `Prophet.make_future_dataframe`, nous créons un dataframe qui contiendra toutes les dates de l'historique et s'étendra également dans le futur pour les 30 jours que nous avons omis auparavant.

In [ ]:
future = m.make_future_dataframe(periods=prediction_size)
future.tail(n=3)

Nous prédisons les valeurs avec `Prophet` en passant les dates pour lesquelles nous voulons créer une prévision. Si nous fournissons également les dates historiques (comme dans notre cas), en plus de la prédiction, nous obtiendrons un ajustement dans l'échantillon pour l'historique. Appelons la méthode `predict` du modèle avec notre dataframe `future` en entrée:

In [ ]:
forecast = m.predict(future)
forecast.tail(n=3)

Dans le dataframe résultant, vous pouvez voir de nombreuses colonnes caractérisant la prédiction, y compris les composants de tendance et de saisonnalité ainsi que leurs intervalles de confiance. La prévision elle-même est stockée dans la colonne `yhat`.

La bibliothèque Prophet possède ses propres outils de visualisation intégrés qui nous permettent d'évaluer rapidement le résultat.

Tout d'abord, il existe une méthode appelée `Prophet.plot` qui trace tous les points de la prévision:

In [ ]:
m.plot(forecast);

Ce graphique n'a pas l'air très informatif. La seule conclusion définitive que nous pouvons tirer ici est que le modèle a traité de nombreux points de données comme des valeurs aberrantes.

La deuxième fonction `Prophet.plot_components` pourrait être beaucoup plus utile dans notre cas. Il nous permet d'observer différentes composantes du modèle séparément: tendance, saisonnalité annuelle et hebdomadaire. De plus, si vous fournissez des informations sur les vacances et les événements à votre modèle, elles seront également affichées dans ce graphique.

Essayons-le:

In [ ]:
m.plot_components(forecast);

Comme vous pouvez le voir sur le graphique des tendances, Prophet a fait du bon travail en adaptant la croissance accélérée des nouveaux messages à la fin de 2016. Le graphique de la saisonnalité hebdomadaire conduit à la conclusion qu'il y a généralement moins de nouveaux messages le samedi et le dimanche que le les autres jours de la semaine. Dans le graphique de saisonnalité annuelle, il y a une baisse importante le jour de Noël.

### 3.5 Évaluation de la qualité des prévisions

Évaluons la qualité de l'algorithme en calculant les mesures d'erreur pour les 30 derniers jours que nous avons prédits. Pour cela, nous aurons besoin des observations $y_i$ et des valeurs prédites correspondantes $\hat{y}_i$.

Examinons l'objet `forecast` que la bibliothèque a créé pour nous:

In [ ]:
print(", ".join(forecast.columns))

Nous pouvons voir que cette base de données contient toutes les informations dont nous avons besoin, à l'exception des valeurs historiques. Nous devons joindre l'objet `forecast` avec les valeurs réelles `y` de l'ensemble de données d'origine `df`. Pour cela nous allons définir une helper fonction que nous réutiliserons plus tard:

In [ ]:
def make_comparison_dataframe(historical, forecast):
    """Join the history with the forecast.
    
       The resulting dataset will contain columns 'yhat', 'yhat_lower', 'yhat_upper' and 'y'.
    """
    return forecast.set_index("ds")[["yhat", "yhat_lower", "yhat_upper"]].join(
        historical.set_index("ds")
    )

Appliquons cette fonction à notre dernière prévision:

In [ ]:
cmp_df = make_comparison_dataframe(df, forecast)
cmp_df.tail(n=3)

Nous allons également définir une helper fonction que nous utiliserons pour évaluer la qualité de nos prévisions avec les mesures d'erreur MAPE et MAE:

In [ ]:
def calculate_forecast_errors(df, prediction_size):
    """Calculate MAPE and MAE of the forecast.
    
       Args:
           df: joined dataset with 'y' and 'yhat' columns.
           prediction_size: number of days at the end to predict.
    """

    # Make a copy
    df = df.copy()

    # Now we calculate the values of e_i and p_i according to the formulas given in the article above.
    df["e"] = df["y"] - df["yhat"]
    df["p"] = 100 * df["e"] / df["y"]

    # Recall that we held out the values of the last `prediction_size` days
    # in order to predict them and measure the quality of the model.

    # Now cut out the part of the data which we made our prediction for.
    predicted_part = df[-prediction_size:]

    # Define the function that averages absolute error values over the predicted part.
    error_mean = lambda error_name: np.mean(np.abs(predicted_part[error_name]))

    # Now we can calculate MAPE and MAE and return the resulting dictionary of errors.
    return {"MAPE": error_mean("p"), "MAE": error_mean("e")}

Utilisons notre fonction:

In [ ]:
for err_name, err_value in calculate_forecast_errors(cmp_df, prediction_size).items():
    print(err_name, err_value)

En conséquence, l'erreur relative de notre prévision (MAPE) est d'environ 22,72%, et en moyenne notre modèle est erroné de 70,45 posts (MAE).

### 3.6 Visualisation

Créons notre propre visualisation du modèle construit par Prophet. Il comprendra les valeurs réelles, les prévisions et les intervalles de confiance.

Premièrement, nous allons tracer les données sur une période de temps plus courte pour rendre les points de données plus faciles à distinguer. Deuxièmement, nous ne montrerons les performances du modèle que pour la période que nous avons prévue, c'est-à-dire les 30 derniers jours. Il semble que ces deux mesures devraient nous donner un graphique plus lisible.

Troisièmement, nous utiliserons `Plotly` pour rendre notre graphique interactif, ce qui est idéal pour l'exploration.

Nous définirons notre propre helper fonction `show_forecast` et l'appellerons (pour en savoir plus sur son fonctionnement, veuillez vous référer aux commentaires dans le code et la [documentation](https://plot.ly/python/)):

In [ ]:
def show_forecast(cmp_df, num_predictions, num_values, title):
    """Visualize the forecast."""

    def create_go(name, column, num, **kwargs):
        points = cmp_df.tail(num)
        args = dict(name=name, x=points.index, y=points[column], mode="lines")
        args.update(kwargs)
        return go.Scatter(**args)

    lower_bound = create_go(
        "Lower Bound",
        "yhat_lower",
        num_predictions,
        line=dict(width=0),
        marker=dict(color="444"),
    )
    upper_bound = create_go(
        "Upper Bound",
        "yhat_upper",
        num_predictions,
        line=dict(width=0),
        marker=dict(color="444"),
        fillcolor="rgba(68, 68, 68, 0.3)",
        fill="tonexty",
    )
    forecast = create_go(
        "Forecast", "yhat", num_predictions, line=dict(color="rgb(31, 119, 180)")
    )
    actual = create_go("Actual", "y", num_values, marker=dict(color="red"))

    # In this case the order of the series is important because of the filling
    data = [lower_bound, upper_bound, forecast, actual]

    layout = go.Layout(yaxis=dict(title="Posts"), title=title, showlegend=False)
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, show_link=False)


show_forecast(cmp_df, prediction_size, 100, "New posts on Medium")

À première vue, la prédiction des valeurs moyennes par notre modèle semble raisonnable. La valeur élevée de MAPE que nous avons obtenue ci-dessus peut s'expliquer par le fait que le modèle n'a pas réussi à saisir l'amplitude croissante de pic-à-pic (peak-to-peak) d'une faible saisonnalité. 

En outre, nous pouvons conclure du graphique ci-dessus que de nombreuses valeurs réelles se trouvent en dehors de l'intervalle de confiance. Prophet peut ne pas convenir aux séries chronologiques avec une variance instable, du moins lorsque les paramètres par défaut sont utilisés. Nous allons essayer de résoudre ce problème en appliquant une transformation à nos données.

## 4. Transformation Box-Cox

Jusqu'à présent, nous avons utilisé Prophet avec les paramètres par défaut et les données d'origine. Nous laisserons les paramètres du modèle seuls. Mais malgré cela, nous avons encore des progrès à faire. Dans cette section, nous appliquerons la [Box–Cox transformation](http://onlinestatbook.com/2/transformations/box-cox.html) à notre série originale. Voyons où cela nous mènera.

Quelques mots sur cette transformation. Il s'agit d'une transformation de données monotone qui peut être utilisée pour stabiliser la variance. Nous utiliserons la transformation Box-Cox à un paramètre, qui est définie par l'expression suivante:

$$
\begin{equation}
  boxcox^{(\lambda)}(y_{i}) = \begin{cases}
    \frac{\displaystyle y_{i}^{\lambda} - 1}{\displaystyle \lambda} &, \text{if $\lambda \neq 0$}.\\
    ln(y_{i}) &, \text{if $\lambda = 0$}.
  \end{cases}
\end{equation}
$$

Nous devrons implémenter l'inverse de cette fonction afin de pouvoir restaurer l'échelle de données d'origine. Il est facile de voir que l'inverse est défini comme:

$$
\begin{equation}
  invboxcox^{(\lambda)}(y_{i}) = \begin{cases}
    e^{\left (\frac{\displaystyle ln(\lambda y_{i} + 1)}{\displaystyle \lambda} \right )} &, \text{if $\lambda \neq 0$}.\\
    e^{y_{i}} &, \text{if $\lambda = 0$}.
  \end{cases}
\end{equation}
$$

La fonction correspondante en Python est implémentée comme suit:

In [ ]:
def inverse_boxcox(y, lambda_):
    return np.exp(y) if lambda_ == 0 else np.exp(np.log(lambda_ * y + 1) / lambda_)

Tout d'abord, nous préparons notre jeu de données en définissant son index:

In [ ]:
train_df2 = train_df.copy().set_index("ds")

Ensuite, nous appliquons la fonction `stats.boxcox` de` Scipy`, qui applique la transformation Box – Cox. Dans notre cas, il renverra deux valeurs. La première est la série transformée et la seconde est la valeur trouvée de $\lambda$ qui est optimale en termes de maximum de log-vraisemblance (maximum log-likelihood):

In [ ]:
train_df2["y"], lambda_prophet = stats.boxcox(train_df2["y"])
train_df2.reset_index(inplace=True)

Nous créons un nouveau modèle `Prophet` et répétons le cycle d'ajustement de prévision que nous avons déjà fait ci-dessus:

In [ ]:
m2 = Prophet()
m2.fit(train_df2)
future2 = m2.make_future_dataframe(periods=prediction_size)
forecast2 = m2.predict(future2)

À ce stade, nous devons inverser la transformation de Box – Cox avec notre fonction inverse et la valeur connue de $\lambda$:

In [ ]:
for column in ["yhat", "yhat_lower", "yhat_upper"]:
    forecast2[column] = inverse_boxcox(forecast2[column], lambda_prophet)

Ici, nous allons réutiliser nos outils pour faire le dataframe de comparaison et calculer les erreurs:

In [ ]:
cmp_df2 = make_comparison_dataframe(df, forecast2)
for err_name, err_value in calculate_forecast_errors(cmp_df2, prediction_size).items():
    print(err_name, err_value)

On peut donc affirmer avec certitude que la qualité du modèle s'est améliorée. 

Enfin, tracons nos performances précédentes avec les derniers résultats côte à côte. Notez que nous utilisons `prediction_size` pour le troisième paramètre afin de zoomer sur l'intervalle prévu:

In [ ]:
show_forecast(cmp_df, prediction_size, 100, "No transformations")
show_forecast(cmp_df2, prediction_size, 100, "Box–Cox transformation")

Nous voyons que la prévision des changements hebdomadaires dans le deuxième graphique est beaucoup plus proche des valeurs réelles maintenant.

## 5. Résumé

Nous avons jeté un coup d'œil à *Prophet*, une bibliothèque de prévisions open source spécifiquement destinée aux séries chronologiques commerciales. Nous avons également effectué des exercices pratiques de prévision des séries chronologiques.

Comme nous l'avons vu, la bibliothèque Prophet ne fait pas de merveilles et ses prédictions prêtes à l'emploi ne sont pas [idéales](https://en.wikipedia.org/wiki/No_free_lunch_in_search_and_optimization). Il appartient toujours au data scientist d'explorer les résultats des prévisions, d'ajuster les paramètres du modèle et de transformer les données si nécessaire.

Toutefois, cette bibliothèque est conviviale et facilement personnalisable. La seule possibilité de prendre en compte les jours anormaux connus de l'analyste à l'avance peut faire la différence dans certains cas

Dans l'ensemble, la bibliothèque Prophet vaut la peine de faire partie de votre boîte à outils analytiques.

## 6. Références

- Official [Prophet repository](https://github.com/facebookincubator/prophet) on GitHub.
- Official [Prophet documentation](https://facebookincubator.github.io/prophet/docs/quick_start.html).
- Sean J. Taylor, Benjamin Letham ["Forecasting at scale"](https://facebookincubator.github.io/prophet/static/prophet_paper_20170113.pdf) — scientific paper explaining the algorithm which lays the foundation of `Prophet`.
- [Forecasting Website Traffic Using Facebook’s Prophet Library](http://pbpython.com/prophet-overview.html) — `Prophet` overview with an example of website traffic forecasting.
- Rob J. Hyndman, George Athanasopoulos ["Forecasting: principles and practice"](https://www.otexts.org/fpp) – a very good online book about time series forecasting.